# crystal_space_group Calculation

**Lucas M. Hale**, [lucas.hale@nist.gov](mailto:lucas.hale@nist.gov?Subject=ipr-demo), *Materials Science and Engineering Division, NIST*.

**Chandler A. Becker**, [chandler.becker@nist.gov](mailto:chandler.becker@nist.gov?Subject=ipr-demo), *Office of Data and Informatics, NIST*.

**Zachary T. Trautt**, [zachary.trautt@nist.gov](mailto:zachary.trautt@nist.gov?Subject=ipr-demo), *Materials Measurement Science Division, NIST*.

Version: 2018-06-24

[Disclaimers](http://www.nist.gov/public_affairs/disclaimer.cfm) 

## Introduction

The crystal_space_group calculation characterizes a bulk atomic system (configuration) by determining its space group by evaluating symmetry elements of the box dimensions and atomic position.  This is useful for analyzing relaxed systems to determine if they have transformed to a different crystal structure.  An ideal unit cell based on the identified space group and the system's box dimensions is also generated.

__Additional dependencies__: This calculation uses the Python spglib package to perform the space group analysis.  Installing the package is required for this calculation.

__Disclaimer #1__: The results are sensitive to the symmetryprecision parameter as it defines the tolerance for identifying which atomic positions and box dimensions are symmetrically equivalent.  A small symmetryprecision value may be able to differentiate between ideal and distorted crystals, but it will cause the calculation to fail if smaller than the variability in the associated system properties.

## Method and Theory

The calculation relies on the spglib Python package, which itself is a wrapper around the spglib library.  The library analyzes an atomic configuration to determine symmetry elements within a precision tolerance for the atomic positions and the box dimensions.  It also contains a database of information related to the different space groups.

More information can be found at

Python spglib

spglib

## Demonstration

### 1. Setup

#### 1.1. Library imports

Import libraries needed by the calculation. The external libraries used are:

- [numpy](http://www.numpy.org/)

- [DataModelDict](https://github.com/usnistgov/DataModelDict)

- [atomman](https://github.com/usnistgov/atomman)

- [iprPy](https://github.com/usnistgov/iprPy)

- [spglib](https://atztogo.github.io/spglib/python-spglib.html)

In [1]:
# Standard library imports
from __future__ import division, absolute_import, print_function
import os
import sys
import uuid
import glob
import shutil
import datetime
from copy import deepcopy

# http://www.numpy.org/
import numpy as np

# https://atztogo.github.io/spglib/python-spglib.html
import spglib

# https://github.com/usnistgov/DataModelDict 
from DataModelDict import DataModelDict as DM

# https://github.com/usnistgov/atomman 
import atomman as am
import atomman.lammps as lmp
import atomman.unitconvert as uc

# https://github.com/usnistgov/iprPy
import iprPy

#### 1.2. Default calculation setup

In [2]:
# Specify calculation style
calc_style = 'crystal_space_group'

# If workingdir is already set, then do nothing (already in correct folder)
try:
    workingdir = workingdir

# Change to workingdir if not already there
except:
    workingdir = os.path.join(os.getcwd(), 'calculationfiles', calc_style)
    if not os.path.isdir(workingdir):
        os.mkdir(workingdir)
    os.chdir(workingdir)

# Default iprPy library directory
librarydir = os.path.join(iprPy.rootdir, '..', 'library')

### 2. Assign values for the calculation's run parameters

#### 2.1. Load initial unit cell system

- __prototype_name__ gives the name of the crystal_prototype reference record in the iprPy library to load. 

- __symbols__ is a list of the potential's elemental model symbols to associate with the unique atom types of the loaded system. 

- __box_parameters__ is a list of the a, b, c lattice constants to assign to the loaded file.

- __load_file__ gives the path to the atomic configuration file to load for the ucell system.  Here, this is generated automatically using prototype_name and librarydir.

- __load_style__ specifies the format of load_file.  Here, this is automatically set for crystal_prototype records.

- __load_options__ specifies any other keyword options for properly loading the load_file.  Here, this is automatically set for crystal_prototype records.

- __ucell__ is an atomman.System representing a fundamental unit cell of the system (required).  Here, this is generated using the load_* parameters and symbols.

In [3]:
prototype_name = 'A1--Cu--fcc'
symbols = ['Ni']
box_parameters = uc.set_in_units([3.52, 3.52, 3.52], 'angstrom')

# Define load_file using librarydir and prototype_name
load_file = os.path.join(librarydir, 'crystal_prototype', prototype_name+'.json')

# Define load_style and load_options for crystal_prototype records
load_style = 'system_model'
load_options = {}

# Create ucell by loading prototype record
ucell = am.load(load_style, load_file, symbols=symbols, **load_options)

# Rescale ucell using box_parameters
ucell.box_set(a=box_parameters[0], b=box_parameters[1], c=box_parameters[2], scale=True)

print(ucell)

avect =  [ 3.520,  0.000,  0.000]
bvect =  [ 0.000,  3.520,  0.000]
cvect =  [ 0.000,  0.000,  3.520]
origin = [ 0.000,  0.000,  0.000]
natoms = 4
natypes = 1
symbols = ('Ni',)
pbc = [ True  True  True]
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |   0.000 |   0.000 |   0.000
      1 |       1 |   0.000 |   1.760 |   1.760
      2 |       1 |   1.760 |   0.000 |   1.760
      3 |       1 |   1.760 |   1.760 |   0.000


#### 2.5. Specify calculation-specific run parameters

- __symmetryprecision__ is a precision tolerance used for the atomic positions and box dimensions for determining symmetry elements.  Default value is '0.01 angstrom'.

- __primitivecell__ is a boolean flag indicating if the returned unit cell is to be primitive (True) or conventional (False).  Default value is False.

- __idealcell__ is a boolean flag indicating if the box dimensions and atomic positions are to be idealized based on the space group (True) or averaged based on their actual values (False).  Default value is True.

In [4]:
symmetryprecision = uc.set_in_units(0.01, 'angstrom')
primitivecell = True
idealcell = True

### 3. Define calculation function(s) and generate template LAMMPS script(s)

#### 3.1. crystal_space_group()

In [5]:
def crystal_space_group(system, symprec=1e-5, to_primitive=False,
                        no_idealize=False):
    """
    Uses spglib to evaluate space group information for a given system.
    
    Parameters
    ----------
    system : atomman.System
        The system to analyze.
    symprec : float
        Absolute length tolerance to use in identifying symmetry of atomic
        sites and system boundaries.
    to_primitive : bool
        Indicates if the returned unit cell is conventional (False) or
        primitive (True). Default value is False.
    no_idealize : bool
        Indicates if the atom positions in the returned unit cell are averaged
        (True) or idealized based on the structure (False).  Default value is
        False.
    
    Returns
    -------
    dict
        Results dictionary containing space group information and an associated
        unit cell system.
    """
    # Identify the standardized unit cell representation
    ucell = spglib.standardize_cell(system.dump('spglib_cell'),
                                    to_primitive=to_primitive,
                                    no_idealize=no_idealize, symprec=symprec)
    
    # Convert back to atomman systems and normalize
    ucell = am.load('spglib_cell', ucell, symbols=system.symbols)
    ucell.atoms.pos -= ucell.atoms.pos[0]
    ucell = ucell.normalize()
    
    # Get space group metadata
    sym_data = spglib.get_symmetry_dataset(ucell.dump('spglib_cell'))
    spg_type = spglib.get_spacegroup_type(sym_data['hall_number'])
    
    # Generate Pearson symbol
    if spg_type['number'] <= 2:
        crystalclass = 'a'
    elif spg_type['number'] <= 15:
        crystalclass = 'm'
    elif spg_type['number'] <= 74:
        crystalclass = 'o'
    elif spg_type['number'] <= 142:
        crystalclass = 't'
    elif spg_type['number'] <= 194:
        crystalclass = 'h'
    else:
        crystalclass = 'c'
    
    latticetype = spg_type['international'][0]
    if latticetype in ['A', 'B']:
        latticetype = 'C'
    
    natoms = str(ucell.natoms)
    pearson = crystalclass + latticetype + natoms
    
    # Return results
    results_dict = spg_type
    results_dict['ucell'] = ucell
    results_dict['hall_number'] = sym_data['hall_number']
    results_dict['wyckoffs'] = sym_data['wyckoffs']
    results_dict['pearson'] = pearson
    
    return results_dict
    


### 4. Run calculation function(s)

In [6]:
results_dict = crystal_space_group(ucell,
                                   symprec=symmetryprecision,
                                   to_primitive=primitivecell,
                                   no_idealize=not idealcell)

### 5. Report results

#### 5.1. Display space group information

In [7]:
for key in results_dict.keys():
    print(key)
    print(results_dict[key])
    print()

number
225

international_short
Fm-3m

international_full
F 4/m -3 2/m

international
F m -3 m

schoenflies
Oh^5

hall_symbol
-F 4 2 3

choice


pointgroup_schoenflies
m-3m

pointgroup_international
Oh

arithmetic_crystal_class_number
72

arithmetic_crystal_class_symbol
m-3mF

ucell
avect =  [ 2.489,  0.000,  0.000]
bvect =  [ 1.245,  2.156,  0.000]
cvect =  [ 1.245,  0.719,  2.032]
origin = [ 0.000,  0.000,  0.000]
natoms = 1
natypes = 1
symbols = ('Ni',)
pbc = [ True  True  True]
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |   0.000 |   0.000 |   0.000

hall_number
523

wyckoffs
['a']

pearson
cF1

